In [2]:
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

from peft import get_peft_model, LoraConfig, TaskType
import os

/home/dkham/Documents/tmp/AI_project/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_old = load_dataset("KomeijiForce/Text2Emoji")

ROW_NUMBER = dataset_old.num_rows["train"]
VALIDATION_SIZE = (ROW_NUMBER * 20) // 100

In [3]:
sliced_train_dataset = dataset_old["train"].select(range(ROW_NUMBER))

dataset = dataset_old.copy()
dataset["train"] = sliced_train_dataset
dataset

{'train': Dataset({
     features: ['text', 'emoji', 'topic'],
     num_rows: 503687
 })}

In [4]:
dct = {"text": [], "emoji": [], "topic": []}
with open("./data/gpt_translate_2.txt", "r", encoding="utf-8") as file:
    for line in file:
        inx = line.rfind(",")
        text, emoji = line[:inx], line[inx + 1 :]
        dct["text"].append(text.strip())
        dct["emoji"].append(emoji.strip())
        dct["topic"].append("None")

In [5]:
extension_data = Dataset.from_dict(dct)
extended_dataset = concatenate_datasets([extension_data, dataset['train']])
dataset['train'] = extended_dataset
dataset

{'train': Dataset({
     features: ['text', 'emoji', 'topic'],
     num_rows: 555314
 })}

In [6]:
NEW_ROW = dataset["train"].num_rows
VALIDATION = (NEW_ROW * 20) // 100

In [3]:
tokenizer_input = GPT2Tokenizer.from_pretrained("gpt2")

/home/dkham/Documents/tmp/AI_project/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def transform_features(example):
    if not all(value is not None for value in example.values()):
        return {
            "full_text": None,
            "prompt_len": 0,
        }
    prompt = example["text"].strip() + " ->"
    target = example["emoji"].strip()
    full_text = prompt + " " + target
    return {
        "full_text": full_text,
        "prompt_len": len(tokenizer_input(full_text)["input_ids"]),
    }


transformed_dataset = dataset["train"].map(
    transform_features, remove_columns=["text", "emoji", "topic"]
)


In [9]:
train_test_split = transformed_dataset.shuffle(seed=42).train_test_split(
    test_size=VALIDATION
)

final_data = DatasetDict(
    {
        "train": train_test_split["train"].select(range(NEW_ROW - VALIDATION)),
        "validation": train_test_split["test"],
    }
)

print(final_data)

DatasetDict({
    train: Dataset({
        features: ['full_text', 'prompt_len'],
        num_rows: 444252
    })
    validation: Dataset({
        features: ['full_text', 'prompt_len'],
        num_rows: 111062
    })
})


In [10]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [11]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
tokenizer_input.pad_token = tokenizer_input.eos_token
model.config.pad_token_id = tokenizer_input.pad_token_id

In [13]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)

/home/dkham/Documents/tmp/AI_project/env/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [14]:
model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


In [15]:
final_data = final_data.filter(
    lambda example: all(value is not None for value in example.values())
)

Filter: 100%|██████████| 111062/111062 [00:01<00:00, 109039.61 examples/s]


In [16]:
def tokenize_dataset(sample):
    max_length = 64
    tokenized = tokenizer_input(
        sample["full_text"],
        padding="max_length",
        max_length=max_length,
        truncation=True,
        return_attention_mask=False,
    )["input_ids"]

    return {"tokenized": tokenized}


In [17]:
shuffled_dataset = final_data.shuffle(seed=42)
tokenized_dataset = shuffled_dataset.map(tokenize_dataset, batched=True)

Map: 100%|██████████| 111060/111060 [00:22<00:00, 4976.34 examples/s]


In [18]:
training_args = TrainingArguments(
    output_dir="./gpt_finetuned",
    eval_strategy="steps",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=32,
    logging_steps=1000,
    save_steps=500,
    eval_steps=8000,
    warmup_steps=2000,
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True,
    logging_first_step=True,
    num_train_epochs=3,
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_input,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]["tokenized"],
    eval_dataset=tokenized_dataset["validation"]["tokenized"],
    tokenizer=tokenizer_input,
    data_collator=data_collator,
)


trainer.train()

  0%|          | 0/10413 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
  0%|          | 1/10413 [00:01<3:01:22,  1.05s/it]

{'loss': 4.4222, 'grad_norm': 0.43945491313934326, 'learning_rate': 2.5000000000000002e-08, 'epoch': 0.0}


 10%|▉         | 1000/10413 [09:39<1:37:54,  1.60it/s]

{'loss': 3.7197, 'grad_norm': 0.3509596586227417, 'learning_rate': 2.5e-05, 'epoch': 0.29}


 19%|█▉        | 2000/10413 [20:10<1:28:32,  1.58it/s]

...

 96%|█████████▌| 10000/10413 [1:34:59<03:36,  1.90it/s]

{'loss': 2.664, 'grad_norm': 0.70386803150177, 'learning_rate': 2.4664210150956854e-06, 'epoch': 2.88}


100%|██████████| 10413/10413 [1:38:37<00:00,  1.76it/s]

{'train_runtime': 5917.4768, 'train_samples_per_second': 225.222, 'train_steps_per_second': 1.76, 'train_loss': 2.8574056418384175, 'epoch': 3.0}


TrainOutput(global_step=10413, training_loss=2.8574056418384175, metrics={'train_runtime': 5917.4768, 'train_samples_per_second': 225.222, 'train_steps_per_second': 1.76, 'total_flos': 4.394457496741478e+16, 'train_loss': 2.8574056418384175, 'epoch': 3.0})

In [5]:
save_directory = f"./model_gpt_{503687}_emoji"

In [ ]:

if not os.path.exists(save_directory):
    os.mkdir(save_directory)
model.save_pretrained(save_directory)

In [6]:
device = "cuda"
model = AutoModelForCausalLM.from_pretrained(save_directory).to(device)

In [17]:
input_text = "Travelling across the globe."
inputs = tokenizer_input(input_text, return_tensors="pt").to(device)

outputs = model.generate(
    inputs.input_ids,
    max_length=40,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer_input.pad_token_id,
)

output_ids = outputs[0].cpu().tolist()
generated_text = tokenizer_input.decode(output_ids, skip_special_tokens=True).strip()
print("Generated Output:")
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Output:
Travelling across the globe. The adrenaline rush is pure bliss! -> 🚶‍♀️🌤️🏃‍♀️💪�
